In [ ]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from pyodesys.symbolic import PartiallySolvedSystem, ScaledSys, symmetricsys, get_logexp
from pycvodes import fpes
from chempy import ReactionSystem, Substance
from chempy.kinetics.ode import get_odesys
sp.init_printing()
%matplotlib inline

In [ ]:
rsys = ReactionSystem.from_string("""
A -> B; 0.04
B + C -> A + C; 1e4
2 B -> B + C; 3e7
""", substance_factory=lambda formula: Substance(formula, composition={1: 1}))

In [ ]:
rsys.composition_balance_vectors()

In [ ]:
dep_scaling = 1e6
orisys, extra = get_odesys(rsys, description='original')
scaledsys = ScaledSys.from_other(orisys, dep_scaling=dep_scaling, description='scaled')

In [ ]:
orisys.linear_invariants, orisys.names, orisys.latex_names

In [ ]:
psysA = PartiallySolvedSystem.from_linear_invariants(orisys, ['A'], description='A')
psysB = PartiallySolvedSystem.from_linear_invariants(orisys, ['B'], description='B')
psysC = PartiallySolvedSystem.from_linear_invariants(orisys, ['C'], description='C')

In [ ]:
psysA.dep, psysB.dep, psysC.dep

In [ ]:
linsystems = [orisys, psysA, psysB, psysC]

def integrate_systems(systems, **kwargs):
    return [odesys.integrate(
        1e14, {'A': 1, 'B': 0, 'C': 0}, integrator='cvode',
        record_rhs_xvals=True, record_jac_xvals=True, record_order=True, record_fpe=True,
        atol=1e-6, rtol=1e-6, **kwargs) for odesys in systems]

In [ ]:
linresults = integrate_systems(linsystems, nsteps=5000)

In [ ]:
def plot_result(odesys, res, ax=None, post_proc=None):
    vk = 'steps rhs_xvals jac_xvals'.split()  # 'fe_underflow fe_overflow fe_invalid fe_divbyzero'
    _ = res.plot(xscale='log', yscale='log', ax=ax, info_vlines_kw=dict(vline_keys=vk, post_proc=post_proc))
    ax.legend(loc='best')
    ax.set_title(odesys.description + (
            ' (%d steps, %d rhs evals., %.5g s CPUt)' % (
                res.info['n_steps'], res.info['nfev'], res.info['time_cpu'])) +
                    '' if res.info['success'] else ' - failed!')

In [ ]:
def plot_results(systems, results, **kwargs):
    fig, axes = plt.subplots(2, 2, figsize=(14, 14))
    for idx, (odesys, res) in enumerate(zip(systems, results)):
        plot_result(odesys, res, ax=axes.flat[idx], **kwargs)

In [ ]:
plot_results(linsystems, linresults)

In [ ]:
LogLogSys = symmetricsys(get_logexp(1, 1e-20, b2=0), get_logexp(1, 1e-12, b2=0), exprs_process_cb=lambda exprs: [
                         sp.powsimp(expr.expand(), force=True) for expr in exprs])
logsystems = [LogLogSys.from_other(ls) for ls in linsystems]

In [ ]:
logresults = integrate_systems(logsystems, nsteps=15000, return_on_error=True)

In [ ]:
lnds = np.log(dep_scaling)
plot_results(logsystems, logresults, post_proc=lambda x: np.exp(x - lnds))

In [ ]:
logsystems[0].exprs